In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
df = pd.read_csv(r"C:\Users\sinha\Music\Music Recommadation System on Machine Learning\Data\ex.csv")

In [3]:
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)

In [4]:
df['User-Rating'] = df['User-Rating'].str[:3]

In [5]:
df['Album/Movie'] = df['Album/Movie'].str.replace(' ', '')
df['Singer/Artists'] = df['Singer/Artists'].str.replace(',', '  ')

In [6]:
df['tags'] = (df['Singer/Artists'] + ' ' + df['Genre'] + ' ' + 
              df['Album/Movie'] + ' ' + df['User-Rating']).str.lower()

In [7]:
new_df = df[['Song-Name', 'tags']].rename(columns={'Song-Name': 'title'})

In [8]:
cv = CountVectorizer(max_features=2000)
vectors = cv.fit_transform(new_df['tags']).toarray()

In [9]:
similarity = cosine_similarity(vectors)

In [10]:
# Function to recommend songs
def recommend(music, new_df=new_df, similarity=similarity):
    try:
        # Find the index of the given song
        music_index = new_df[new_df['title'] == music].index[0]
        distances = similarity[music_index]
        
        # Sort the songs based on similarity scores and get the top 5 recommendations
        music_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
        recommended_songs = [new_df.iloc[i[0]].title for i in music_list]
        
        return recommended_songs
    except IndexError:
        return ["No recommendations found. Try a different song."]

In [11]:
pickle.dump(new_df, open('musicrec.pkl', 'wb'))
pickle.dump(similarity, open('similarities.pkl', 'wb'))
